In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
skater_folder = "../skaters/"
skater_files = os.listdir(skater_folder)
team_folder = "../tmstats/"
team_files = os.listdir(team_folder)